In [ ]:
# !conda install -y -c conda-forge pyspark
# !conda install -y -c conda-forge black
# !conda install -y -c conda-forge pip
# !pip install delta-spark==2.1.1
# !sudo wget --user jovyan --password jovyan https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.1.0/delta-core_2.12-2.1.0.jar -P $SPARK_HOME/jars/


In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, DataFrame
from pyspark.conf import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from functools import reduce
import json
import glob
import tempfile
from delta import *

builder = (
    SparkSession.builder.master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# spark = SparkSession.builder \
#     .master("local") \
#     .getOrCreate()

In [ ]:
all_files = all_files = glob.glob("../data/AssignmentData/*.csv")
all_files

In [ ]:
asset_columns = {'AS1':DateType(),
'AS2':StringType(),
'AS3':StringType(),
'AS4':StringType(),
'AS5':StringType(),
'AS6':StringType(),
'AS7':StringType(),
'AS8':StringType(),
'AS15':StringType(),
'AS16':StringType(),
'AS17':StringType(),
'AS18':StringType(),
'AS19':DateType(),
'AS20':DateType(),
'AS21':StringType(),
'AS22':StringType(),
'AS23':BooleanType(),
'AS24':StringType(),
'AS25':StringType(),
'AS26':StringType(),
'AS27':DoubleType(),
'AS28':DoubleType(),
'AS29':BooleanType(),
'AS30':DoubleType(),
'AS31':DateType(),
'AS32':StringType(),
'AS33':StringType(),
'AS34':StringType(),
'AS35':StringType(),
'AS36':StringType(),
'AS37':DoubleType(),
'AS38':DoubleType(),
'AS39':DoubleType(),
'AS40':DoubleType(),
'AS41':DoubleType(),
'AS42':StringType(),
'AS43':StringType(),
'AS44':DoubleType(),
'AS45':StringType(),
'AS50':DateType(),
'AS51':DateType(),
'AS52':StringType(),
'AS53':BooleanType(),
'AS54':DoubleType(),
'AS55':DoubleType(),
'AS56':DoubleType(),
'AS57':StringType(),
'AS58':StringType(),
'AS59':StringType(),
'AS60':DoubleType(),
'AS61':DoubleType(),
'AS62':StringType(),
'AS63':DoubleType(),
'AS64':DoubleType(),
'AS65':StringType(),
'AS66':DoubleType(),
'AS67':DateType(),
'AS68':StringType(),
'AS69':DoubleType(),
'AS70':DateType(),
'AS71':DateType(),
'AS80':DoubleType(),
'AS81':DoubleType(),
'AS82':DoubleType(),
'AS83':StringType(),
'AS84':StringType(),
'AS85':DoubleType(),
'AS86':DoubleType(),
'AS87':DateType(),
'AS88':DoubleType(),
'AS89':StringType(),
'AS90':DoubleType(),
'AS91':DateType(),
'AS92':StringType(),
'AS93':DoubleType(),
'AS94':StringType(),
'AS100':DoubleType(),
'AS101':DoubleType(),
'AS102':DoubleType(),
'AS103':DoubleType(),
'AS104':DoubleType(),
'AS105':DoubleType(),
'AS106':DoubleType(),
'AS107':DoubleType(),
'AS108':DoubleType(),
'AS109':DoubleType(),
'AS110':DoubleType(),
'AS111':StringType(),
'AS112':DateType(),
'AS115':DoubleType(),
'AS116':DoubleType(),
'AS117':DoubleType(),
'AS118':DoubleType(),
'AS119':DoubleType(),
'AS120':DoubleType(),
'AS121':BooleanType(),
'AS122':BooleanType(),
'AS123':StringType(),
'AS124':DateType(),
'AS125':DoubleType(),
'AS126':DoubleType(),
'AS127':DateType(),
'AS128':DoubleType(),
'AS129':StringType(),
'AS130':DateType(),
'AS131':BooleanType(),
'AS132':DoubleType(),
'AS133':DateType(),
'AS134':DateType(),
'AS135':DoubleType(),
'AS136':DoubleType(),
'AS137':DateType(),
'AS138':DoubleType()}

In [ ]:
# # Find List columns and change numerical value to literal
# mapper_dict={
#     "Obligor Legal Form / Business Type":{1:"Public Company",2:"Limited Company",3:"Partnership",4:"Individul",5:"Other"},
#     "Customer Segment":{1:"Medium",2:"Small",3:"Micro",4:"Other"},
#     "Borrower Basel III Segment":{1:"Corporate",2:"SME treated as Corporate",3:"Retail",4:"Other"},
#     "Asset Type":{1: 'Loan',
#                 2: 'Guarantee',
#                 3: 'Promissory Notes',
#                 4: 'Participation Rights',
#                 5: 'Overdraft',
#                 6: 'Letter of Credit',
#                 7: 'Working Capital Facility',
#                 8: 'Other'},
#     "Seniority":{1: 'Senior Secured',
#                 2: 'Senior Unsecured',
#                 3: 'Junior',
#                 4: 'Junior Unsecured',
#                 5: 'Other'},
#     "Purpose":{1: 'Purchase',
#             2: 'Re-mortgage',
#             3: 'Renovation',
#             4: 'Equity release',
#             5: 'Construction Real Estate',
#             6: 'Construction Other',
#             7: 'Debt consolidation',
#             8: 'Re-mortgage with Equity Release',
#             9: 'Re-mortgage on Different Terms',
#             10: 'Combination Mortgage',
#             11: 'Investment Mortgage',
#             12: 'Working Capital',
#             13: 'Other'},
#     "Principal Payment Frequency":{1: 'Monthly',
#                                 2: 'Quarterly',
#                                 3: 'Semi annually',
#                                 4: 'Annual',
#                                 5: 'Bullet',
#                                 6: 'Other'},
#     "Interest Payment Frequency":{1: 'Linear',
#                             2: 'French',
#                             3: 'Fix Amortisation Schedule',
#                             4: 'Bullet',
#                             5: 'Partial Bullet',
#                             6: 'Revolving',
#                             7: 'Other'},
#     "Type of Loan":{1: 'Term', 2: 'Revolving Credit Line', 3: 'Other'},
#     "Payment type":{1: 'Direct Debit', 2: 'Standing Order', 3: 'Cheque', 4: 'Cash', 5: 'Other'},
#     "Interest Rate Type":{1: 'Floating rate loan for life',
#                         2: "Floating rate loan linked to Libor, Euribor, BoE reverting to the Bank's SVR, ECB reverting to Bank’s SVR",
#                         3: 'Fixed rate loan for life',
#                         4: 'Fixed with future periodic resets',
#                         5: 'Fixed rate loan with compulsory future switch to floating',
#                         6: 'Capped',
#                         7: 'Discount',
#                         8: 'Switch Optionality',
#                         9: 'Borrower Swapped',
#                         10: 'Other'},
#     "Current Interest Rate Index":{1: '1 month LIBOR',
#                                 2: '1 month EURIBOR',
#                                 3: '3 month LIBOR',
#                                 4: '3 month EURIBOR',
#                                 5: '6 month LIBOR',
#                                 6: '6 month EURIBOR',
#                                 7: '12 month LIBOR',
#                                 8: '12 month EURIBOR',
#                                 9: 'BoE Base Rate',
#                                 10: 'ECB Base Rate',
#                                 11: 'Standard Variable Rate',
#                                 12: 'Other'},
#     "Interest Reset Period":{1: 'Annual',
#                             2: 'Semi-annual',
#                             3: 'Quarterly',
#                             4: 'Monthly',
#                             5: 'Not apply',
#                             6: 'Other'},
#     "Reason for Default (Basel II definition)":{1: 'Bankruptcy / Insolvency',
#                                             2: 'Failure to Pay',
#                                             3: 'Breach of Terms',
#                                             4: 'Other'},
#     "Recovery Source":{1: 'Bankruptcy / Insolvency',
#                         2: 'Failure to Pay',
#                         3: 'Breach of Terms',
#                         4: 'Other'},
#     "Recovery Source":{1: 'Liquidation of Collateral',
#                         2: 'Enforcement of Guarantees',
#                         3: 'Additional Lending',
#                         4: 'Cash Recoveries',
#                         5: 'Mixed',
#                         6: 'Other'},
# }

In [ ]:
import csv 
list_dfs = []
for csv_f in all_files:
    col_names = []
    content = []
    with open(csv_f,'r') as f:
        for i, line in enumerate(csv.reader(f)):
            if i == 0:
                col_names = line
            elif i == 1:
                continue
            else:
                content.append(line)
        df = spark.createDataFrame(content, col_names)
        list_dfs.append(df)
assets_df = reduce(DataFrame.union, list_dfs)

In [ ]:
def replace_no_data(df):
    # For ND  values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name).startswith("ND"), None)
                                    .otherwise(F.col(col_name)))
    return df

test_1 = replace_no_data(assets_df)

test_1.show()

In [ ]:
def replace_bool_data(df):
    # For  boolean values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name) == "Y", "True")
                                    .when(F.col(col_name) == "N", "False")
                                    .otherwise(F.col(col_name)))
    return df
test_2 = replace_bool_data(test_1)
test_2.show()

In [ ]:
def cast_to_datatype(df, columns):
    for col_name, data_type in columns.items():
        if data_type == BooleanType():
            df=df.withColumn("tmp_col_name", F.col(col_name).contains("True")).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DateType():
           df=df.withColumn("tmp_col_name", F.to_date(F.col(col_name))).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DoubleType():
           df=df.withColumn("tmp_col_name", F.round(F.col(col_name).cast(DoubleType()),2)).drop(col_name).withColumnRenamed("tmp_col_name", col_name) 
    return df

test_3 = cast_to_datatype(test_2, asset_columns)
test_3.printSchema()

In [ ]:
test_3.show(n=5,truncate=False)

In [ ]:
# Unable to use Delta Lake locally due to concurrency errors
# test_3.write.mode("overwrite").format("delta").save("/home/jovyan/work/data/output/bronze/asset_bronze.delta")
test_3.write.parquet("../data/output/bronze/asset_bronze.parquet")

## Attempt to read XML Deal Detail file

In [ ]:
# !conda install -y -c conda-forge lxml

In [ ]:
from lxml import objectify, etree

deal_detail_file = "../data/SMESIT000211100520189/SMESIT000211100520189_Deal_Details.xml"

xml_data = objectify.parse(deal_detail_file)  # Parse XML data
root = xml_data.getroot()  # Root element

data = []
cols = []
for i in range(len(root.getchildren()[1].getchildren()[0].getchildren()[1].getchildren()[0].getchildren())):
    child = root.getchildren()[1].getchildren()[0].getchildren()[1].getchildren()[0].getchildren()[i]
    data.append(child.text)
    cols.append(child.tag.replace("{http://edwin.eurodw.eu/EDServices/2.3}",""))

df = pd.DataFrame(data).T  # Create DataFrame and transpose it
df.columns = cols  # Update column names
df.head()

In [ ]:
df.to_csv("../data/output/bronze/deal_detail.csv", index=False)

## Attempt to work with Amortization Profile data

In [2]:
all_files = all_files = glob.glob("../data/SMESIT000211100520189/*Amortization*.csv")
all_files

['../data/SMESIT000211100520189/SMESIT000211100520189_2018_09_30_Amortization_Profile_634d8f98-3735-4a84-ad0f-424dfe5183f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2018_12_31_Amortization_Profile_b99e7919-09d3-4b57-adf7-7ce8019f37e0.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_03_31_Amortization_Profile_fed414d7-f03d-4640-8d3d-c9b98c6374f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_06_30_Amortization_Profile_10cab6b4-2123-4d2c-a66d-b8ca5550880f.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_09_30_Amortization_Profile_9f45800f-5fdc-4192-bf85-a59345a00162.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_12_31_Amortization_Profile_16ba0fdf-19f9-4efe-b5d5-ba3279997e63.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2020_03_31_Amortization_Profile_f83f3b34-73d8-4337-9ead-a9b01b4766c2.csv']

In [14]:
amortization_columns = {'AS3':StringType(),
                        'AS150':DoubleType(),
                        'AS151':DateType(),
                        'AS1348':DoubleType(),
                        'AS1349':DateType()}
for i in range(152,1348):
    if i % 2 == 0:
        amortization_columns[f'AS{i}']=DoubleType()
    else:
        amortization_columns[f'AS{i}']=DateType()

amortization_columns

{'AS3': StringType(),
 'AS150': DoubleType(),
 'AS151': DateType(),
 'AS1348': DoubleType(),
 'AS1349': DateType(),
 'AS152': DoubleType(),
 'AS153': DateType(),
 'AS154': DoubleType(),
 'AS155': DateType(),
 'AS156': DoubleType(),
 'AS157': DateType(),
 'AS158': DoubleType(),
 'AS159': DateType(),
 'AS160': DoubleType(),
 'AS161': DateType(),
 'AS162': DoubleType(),
 'AS163': DateType(),
 'AS164': DoubleType(),
 'AS165': DateType(),
 'AS166': DoubleType(),
 'AS167': DateType(),
 'AS168': DoubleType(),
 'AS169': DateType(),
 'AS170': DoubleType(),
 'AS171': DateType(),
 'AS172': DoubleType(),
 'AS173': DateType(),
 'AS174': DoubleType(),
 'AS175': DateType(),
 'AS176': DoubleType(),
 'AS177': DateType(),
 'AS178': DoubleType(),
 'AS179': DateType(),
 'AS180': DoubleType(),
 'AS181': DateType(),
 'AS182': DoubleType(),
 'AS183': DateType(),
 'AS184': DoubleType(),
 'AS185': DateType(),
 'AS186': DoubleType(),
 'AS187': DateType(),
 'AS188': DoubleType(),
 'AS189': DateType(),
 'AS190': 

In [4]:
import csv 
list_dfs = []
for csv_f in all_files:
    col_names = []
    content = []
    with open(csv_f,'r') as f:
        for i, line in enumerate(csv.reader(f)):
            if i == 0:
                col_names = line
            elif i == 1:
                continue
            else:
                content.append(list(filter(None,[None if x == '' else x for x in line])))
        df = spark.createDataFrame(content, col_names[:len(content[0])])
        list_dfs.append(df)
amortization_df = reduce(DataFrame.union, list_dfs)
amortization_df = amortization_df.withColumnRenamed("\ufeffAS3","AS3")

In [8]:
def _melt(df, id_vars, value_vars, var_name="FEATURE_NAME", value_name="FEATURE_VALUE"):
    """Convert DataFrame from wide to long format."""
    # Ref:https://stackoverflow.com/a/41673644
    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = F.array(
        *(
            F.struct(F.lit(c).alias(var_name), F.col(c).alias(value_name))
            for c in value_vars
        )
    )
    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", F.explode(_vars_and_vals))
    cols = id_vars + [
        F.col("_vars_and_vals")[x].cast("string").alias(x)
        for x in [var_name, value_name]
    ]
    return _tmp.select(*cols)


In [16]:
amortization_date_columns = [k for k,v in amortization_columns.items() if v == DateType() and k in amortization_df.columns]
amortization_double_columns = [k for k,v in amortization_columns.items() if v == DoubleType() and k in amortization_df.columns]

amortization_date_df =  _melt(amortization_df,id_vars=["AS3"], value_vars=amortization_date_columns,var_name="DATE_COLUMNS", value_name="DATE_VALUE")
amortization_double_df =  _melt(amortization_df,id_vars=["AS3"], value_vars=amortization_double_columns,var_name="DOUBLE_COLUMNS", value_name="DOUBLE_VALUE")
amortization_unpivot_df = amortization_date_df.join(amortization_double_df,on="AS3",how="inner")


In [19]:
amortization_unpivot_df.printSchema()

root
 |-- AS3: string (nullable = true)
 |-- DATE_COLUMNS: string (nullable = false)
 |-- DATE_VALUE: string (nullable = true)
 |-- DOUBLE_COLUMNS: string (nullable = false)
 |-- DOUBLE_VALUE: string (nullable = true)



In [20]:
def cast_to_datatype(df):
    new_df = (
        df.withColumn("tmp_col_name", F.to_date(F.col("DATE_VALUE"))).drop("DATE_VALUE").withColumnRenamed("tmp_col_name", "DATE_VALUE")
            .withColumn("tmp_col_name", F.round(F.col("DOUBLE_VALUE").cast(DoubleType()),2)).drop("DOUBLE_VALUE").withColumnRenamed("tmp_col_name", "DOUBLE_VALUE")
    )
    return new_df
    
test_1 = cast_to_datatype(amortization_unpivot_df)
test_1.printSchema()

root
 |-- AS3: string (nullable = true)
 |-- DATE_COLUMNS: string (nullable = false)
 |-- DOUBLE_COLUMNS: string (nullable = false)
 |-- DATE_VALUE: date (nullable = true)
 |-- DOUBLE_VALUE: double (nullable = true)



In [22]:
test_1.write.parquet("../data/output/bronze/amortization_bronze.parquet")

## Attempt to handle Collateral data

In [2]:
all_files = all_files = glob.glob("../data/SMESIT000211100520189/*Collateral*.csv")
all_files

['../data/SMESIT000211100520189/SMESIT000211100520189_2018_09_30_Collateral_634d8f98-3735-4a84-ad0f-424dfe5183f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2018_12_31_Collateral_b99e7919-09d3-4b57-adf7-7ce8019f37e0.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_03_31_Collateral_fed414d7-f03d-4640-8d3d-c9b98c6374f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_06_30_Collateral_10cab6b4-2123-4d2c-a66d-b8ca5550880f.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_09_30_Collateral_9f45800f-5fdc-4192-bf85-a59345a00162.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_12_31_Collateral_16ba0fdf-19f9-4efe-b5d5-ba3279997e63.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2020_03_31_Collateral_f83f3b34-73d8-4337-9ead-a9b01b4766c2.csv']

In [3]:
collateral_columns = {'CS1':'StringType()',
'CS2':'StringType()',
'CS3':'StringType()',
'CS4':'DoubleType()',
'CS5':'DoubleType()',
'CS6':'StringType()',
'CS7':'BooleanType()',
'CS8':'BooleanType()',
'CS9':'BooleanType()',
'CS10':'DoubleType()',
'CS11':'DateType()',
'CS12':'DateType()',
'CS13':'StringType()',
'CS14':'StringType()',
'CS15':'DoubleType()',
'CS16':'StringType()',
'CS17':'StringType()',
'CS18':'DoubleType()',
'CS19':'DoubleType()',
'CS20':'StringType()',
'CS21':'DoubleType()',
'CS22':'DateType()',
'CS23':'StringType()',
'CS24':'StringType()',
'CS25':'StringType()',
'CS26':'StringType()',
'CS27':'StringType()',
'CS28':'DoubleType()'}

In [8]:
import csv 
list_dfs = []
for csv_f in all_files:
    col_names = []
    content = []
    with open(csv_f,'r') as f:
        for i, line in enumerate(csv.reader(f)):
            if i == 0:
                col_names = line
            elif i == 1:
                continue
            else:
                content.append(line)
        df = spark.createDataFrame(content, col_names)
        list_dfs.append(df)
collateral_df = reduce(DataFrame.union, list_dfs)

col_names = ['CS1',
 'CS2',
 'CS3',
 'CS4',
 'CS5',
 'CS6',
 'CS7',
 'CS8',
 'CS9',
 'CS10',
 'CS11',
 'CS12',
 'CS13',
 'CS14',
 'CS15',
 'CS16',
 'CS17',
 'CS18',
 'CS19',
 'CS20',
 'CS21',
 'CS22',
 'CS23',
 'CS24',
 'CS25',
 'CS26',
 'CS27',
 'CS28']

collateral_df = collateral_df.toDF(*col_names)

In [9]:
def replace_no_data(df):
    # For ND  values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name).startswith("ND"), None)
                                        .when(F.length(col_name)==0, None)
                                    .otherwise(F.col(col_name)))
    return df

test_1 = replace_no_data(collateral_df)

test_1.show(n=100)

+------+--------------------+---+---------+----+---+---+---+---+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|   CS1|                 CS2|CS3|      CS4| CS5|CS6|CS7|CS8|CS9|     CS10|   CS11|   CS12|CS13|CS14|CS15| CS16| CS17|CS18|CS19|CS20|CS21|CS22|CS23|CS24|CS25|CS26|CS27|CS28|
+------+--------------------+---+---------+----+---+---+---+---+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|243985|00100-123-21-0095...|  3|254896.64|0.00| 13|  Y|  Y|  Y|265000.00|2015-08|2018-09|   9|   1|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0006...|  3|     0.00|0.00| 21|  Y|  Y|  Y|     0.00|   null|   null|   9|null|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0028...|  3|     0.00|0.00| 21|  Y|  Y|  Y|     0.00|   null|   null|   9|null|0.00|55100|ITE12|0.00|0.00|   0|0.0

In [10]:
def replace_bool_data(df):
    # For  boolean values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name) == "Y", "True")
                                    .when(F.col(col_name) == "N", "False")
                                    .otherwise(F.col(col_name)))
    return df
test_2 = replace_bool_data(test_1)
test_2.show()

+------+--------------------+---+---------+----+---+----+----+----+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|   CS1|                 CS2|CS3|      CS4| CS5|CS6| CS7| CS8| CS9|     CS10|   CS11|   CS12|CS13|CS14|CS15| CS16| CS17|CS18|CS19|CS20|CS21|CS22|CS23|CS24|CS25|CS26|CS27|CS28|
+------+--------------------+---+---------+----+---+----+----+----+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|243985|00100-123-21-0095...|  3|254896.64|0.00| 13|True|True|True|265000.00|2015-08|2018-09|   9|   1|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0006...|  3|     0.00|0.00| 21|True|True|True|     0.00|   null|   null|   9|null|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0028...|  3|     0.00|0.00| 21|True|True|True|     0.00|   null|   null|   9|null|0.00|55100|ITE12|

In [12]:
def cast_to_datatype(df, columns):
    for col_name, data_type in columns.items():
        if data_type == BooleanType():
            df=df.withColumn("tmp_col_name", F.col(col_name).contains("True")).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DateType():
           df=df.withColumn("tmp_col_name", F.to_date(F.col(col_name))).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DoubleType():
           df=df.withColumn("tmp_col_name", F.round(F.col(col_name).cast(DoubleType()),2)).drop(col_name).withColumnRenamed("tmp_col_name", col_name) 
    return df

test_3 = cast_to_datatype(test_2, collateral_columns)
test_3.show(n=100)

+------+--------------------+---+---------+----+---+----+----+----+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|   CS1|                 CS2|CS3|      CS4| CS5|CS6| CS7| CS8| CS9|     CS10|   CS11|   CS12|CS13|CS14|CS15| CS16| CS17|CS18|CS19|CS20|CS21|CS22|CS23|CS24|CS25|CS26|CS27|CS28|
+------+--------------------+---+---------+----+---+----+----+----+---------+-------+-------+----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+
|243985|00100-123-21-0095...|  3|254896.64|0.00| 13|True|True|True|265000.00|2015-08|2018-09|   9|   1|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0006...|  3|     0.00|0.00| 21|True|True|True|     0.00|   null|   null|   9|null|0.00|55100|ITE12|0.00|0.00|   0|0.00|null|   1| EUR| EUR| EUR| EUR|   1|
|     0|00100-123-31-0028...|  3|     0.00|0.00| 21|True|True|True|     0.00|   null|   null|   9|null|0.00|55100|ITE12|

In [13]:
test_3.write.parquet("../data/output/bronze/collateral_bronze.parquet")

## Attempt to handle Bond Info

In [22]:
all_files = all_files = glob.glob("../data/SMESIT000211100520189/*Bond_Info*.csv")
all_files

['../data/SMESIT000211100520189/SMESIT000211100520189_2018_09_30_Bond_Info_634d8f98-3735-4a84-ad0f-424dfe5183f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2018_12_31_Bond_Info_b99e7919-09d3-4b57-adf7-7ce8019f37e0.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_03_31_Bond_Info_fed414d7-f03d-4640-8d3d-c9b98c6374f2.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_06_30_Bond_Info_10cab6b4-2123-4d2c-a66d-b8ca5550880f.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_09_30_Bond_Info_9f45800f-5fdc-4192-bf85-a59345a00162.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2019_12_31_Bond_Info_16ba0fdf-19f9-4efe-b5d5-ba3279997e63.csv',
 '../data/SMESIT000211100520189/SMESIT000211100520189_2020_03_31_Bond_Info_f83f3b34-73d8-4337-9ead-a9b01b4766c2.csv']

In [23]:
bond_columns = {'BS1':'DateType()',
'BS2':'StringType()',
'BS3':'DoubleType()',
'BS4':'DoubleType()',
'BS5':'BooleanType()',
'BS6':'StringType()',
'BS11':'DoubleType()',
'BS12':'BooleanType()',
'BS13':'DoubleType()',
'BS19':'StringType()',
'BS20':'StringType()'}

In [24]:
import csv 
list_dfs = []
for csv_f in all_files:
    col_names = []
    content = []
    with open(csv_f,'r') as f:
        for i, line in enumerate(csv.reader(f)):
            if i == 0:
                col_names = line
            elif i == 1:
                continue
            else:
                content.append(line)
        df = spark.createDataFrame(content, col_names)
        list_dfs.append(df)
bond_df = reduce(DataFrame.union, list_dfs)

col_names = ['BS1',
 'BS2',
 'BS3',
 'BS4',
 'BS5',
 'BS6',
 'BS11',
 'BS12',
 'BS13',
 'BS19',
 'BS20',
 'BS25',
 'BS26',
 'BS27',
 'BS28',
 'BS29',
 'BS30',
 'BS31',
 'BS32',
 'BS33',
 'BS34',
 'BS35',
 'BS36',
 'BS37',
 'BS38',
 'BS39']
bond_df = bond_df.toDF(*col_names)

In [25]:
def replace_no_data(df):
    # For ND  values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name).startswith("ND"), None)
                                        .when(F.length(col_name)==0, None)
                                    .otherwise(F.col(col_name)))
    return df

test_1 = replace_no_data(bond_df)

test_1.show()

+----------+--------------------+----------+----------+---+---+----+----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|       BS1|                 BS2|       BS3|       BS4|BS5|BS6|BS11|BS12|BS13|                BS19|                BS20|   BS25|        BS26|      BS27|      BS28|BS29|        BS30|        BS31|BS32|      BS33|       BS34|      BS35|BS36|BS37|      BS38|      BS39|
+----------+--------------------+----------+----------+---+---+----+----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|2018-09-30|LANTERNA FINANCE ...|5200000.00|5200000.00|  N|EUR|null|   N|0.00|0039 10 5794204 c...|0039 10 5794204 c...|CLASS A|IT0005333809|2018-10-31|2018-10-31| EUR|260000000.00|260000000.00|   4|   

In [26]:
def replace_bool_data(df):
    # For  boolean values
    for col_name in df.columns:
        df = df.withColumn(col_name, F.when(F.col(col_name) == "Y", "True")
                                    .when(F.col(col_name) == "N", "False")
                                    .otherwise(F.col(col_name)))
    return df
test_2 = replace_bool_data(test_1)
test_2.show()

+----------+--------------------+----------+----------+-----+---+----+-----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|       BS1|                 BS2|       BS3|       BS4|  BS5|BS6|BS11| BS12|BS13|                BS19|                BS20|   BS25|        BS26|      BS27|      BS28|BS29|        BS30|        BS31|BS32|      BS33|       BS34|      BS35|BS36|BS37|      BS38|      BS39|
+----------+--------------------+----------+----------+-----+---+----+-----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|2018-09-30|LANTERNA FINANCE ...|5200000.00|5200000.00|False|EUR|null|False|0.00|0039 10 5794204 c...|0039 10 5794204 c...|CLASS A|IT0005333809|2018-10-31|2018-10-31| EUR|260000000.00|260000000

In [27]:
def cast_to_datatype(df, columns):
    for col_name, data_type in columns.items():
        if data_type == BooleanType():
            df=df.withColumn("tmp_col_name", F.col(col_name).contains("True")).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DateType():
           df=df.withColumn("tmp_col_name", F.to_date(F.col(col_name))).drop(col_name).withColumnRenamed("tmp_col_name", col_name)
        if data_type == DoubleType():
           df=df.withColumn("tmp_col_name", F.round(F.col(col_name).cast(DoubleType()),2)).drop(col_name).withColumnRenamed("tmp_col_name", col_name) 
    return df

test_3 = cast_to_datatype(test_2, bond_columns)
test_3.show()

+----------+--------------------+----------+----------+-----+---+----+-----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|       BS1|                 BS2|       BS3|       BS4|  BS5|BS6|BS11| BS12|BS13|                BS19|                BS20|   BS25|        BS26|      BS27|      BS28|BS29|        BS30|        BS31|BS32|      BS33|       BS34|      BS35|BS36|BS37|      BS38|      BS39|
+----------+--------------------+----------+----------+-----+---+----+-----+----+--------------------+--------------------+-------+------------+----------+----------+----+------------+------------+----+----------+-----------+----------+----+----+----------+----------+
|2018-09-30|LANTERNA FINANCE ...|5200000.00|5200000.00|False|EUR|null|False|0.00|0039 10 5794204 c...|0039 10 5794204 c...|CLASS A|IT0005333809|2018-10-31|2018-10-31| EUR|260000000.00|260000000

In [28]:
test_3.write.parquet("../data/output/bronze/bond_info_bronze.parquet")